<a href="https://colab.research.google.com/github/dimaszuda/LFP_Battery_SoC_Prediction/blob/main/model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import tensorflow as tf
from matplotlib import pyplot as plt
import pandas as pd
from sklearn.preprocessing import MinMaxScaler
from tensorflow import keras
from tensorflow.keras import layers
!pip install keras_tuner
from keras_tuner import GridSearch
from keras_tuner import HyperParameters
from tensorflow.keras.optimizers import SGD, Adam, RMSprop, Adamax
from io import StringIO
import json

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 176.1/176.1 kB 15.3 MB/s eta 0:00:00


In [ ]:
df = pd.ExcelFile('Data Bersih.xlsx').parse()

In [ ]:
df.head()

,CycleID,Step_Type,C_Rate,Voltage(V),Current(mA),Cap(mAh),SoC (%)
0,1,CC_Chg,0.503168,3.32923,899.479,0.00000,0.000
1,1,CC_Chg,0.503113,3.33115,899.380,0.24983,0.014
2,1,CC_Chg,0.503113,3.33414,899.380,0.49966,0.028
3,1,CC_Chg,0.503113,3.33697,899.380,0.74951,0.042
4,1,CC_Chg,0.503168,3.33905,899.479,0.99937,0.056


In [ ]:
columns = ['CycleID',
           'Current(mA)',
           'Voltage(V)',
           'Cap(mAh)']

In [ ]:
X_train = df[columns]
y_train = df['SoC (%)'].values.reshape(-1,1)

In [ ]:
scaler = MinMaxScaler()

In [ ]:
X_train = scaler.fit_transform(X_train)
X_train = pd.DataFrame(X_train, columns=columns)

In [ ]:
y_train = scaler.fit_transform(y_train)
y_train = pd.DataFrame(y_train, columns=['SoC (%)'])

In [ ]:
def build_model(hp):
  model = tf.keras.models.Sequential()
  model.add(layers.Input(shape=(4)))

  for i in range(hp.Choice(name='num_layers', values=[3])):
    model.add(layers.Dense(units=hp.Choice(f'num_of_neurons_{i}', values=[4, 8, 16, 32]), 
          activation='relu'))
  model.add(layers.Dense(1, activation=hp.Choice('activation', values=['sigmoid'])))

  optimizers = hp.Choice('optimizer', values=['adam'])
  learning_rate = hp.Choice('learning_rate', values=[1e-2])

  if optimizers == 'rmsprop':
    optimizer = RMSprop(learning_rate=learning_rate)
  elif optimizers == 'adam':
    optimizer = Adam(learning_rate=learning_rate)
  elif optimizers == 'sgd':
    optimizer = SGD(learning_rate=learning_rate)
  else:
    optimizer = Adamax(learning_rate=learning_rate) 

  model.compile(loss='mae',
                optimizer=optimizer,
                metrics=['mse'])
  
  return model

In [ ]:
tuner = GridSearch(
    build_model,
    objective='loss',
    executions_per_trial=2,
    directory='tuner_model2',
    project_name='Skripsi'
)

In [ ]:
tuner.search_space_summary()
tuner.search(
    X_train,
    y_train,
    epochs=8,
)

NameError: ignored

In [ ]:
trials = tuner.oracle.get_best_trials(num_trials=64)

In [ ]:
trial_id = []
hyperparameters = []
loss = []

In [ ]:
for trial in trials:
  trial_id.append(trial.trial_id)
  hyperparameters.append(trial.hyperparameters.values)
  loss.append(json.dumps(trial.score))

In [ ]:
oke = pd.DataFrame(hyperparameters)

In [ ]:
oke.insert(0, 'trial_id', trial_id)

In [ ]:
oke['loss'] = loss

In [ ]:
simpan = oke.to_excel('Percobaan Model_Skripsi_Dimas_18.xlsx', index=False)

In [ ]:
import files
try:
    files.download('Percobaan Model_Skripsi_Dimas_17.xlsx')
except:
    pass